# Pre-setup and Setup

In [ ]:
! python.exe -m pip install --upgrade pip

In [ ]:
! pip install pandas requests pymongo dot

In [38]:
! pip install pathlib

In [39]:
! pip install python-dotenv

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'error'


  error: subprocess-exited-with-error
  
  × pip subprocess to install backend dependencies did not run successfully.
  │ exit code: 1
  ╰─> [33 lines of output]
           ---------------------------------------- 0.0/145.4 kB ? eta -:--:--
           ------- ----------------------------- 30.7/145.4 kB 435.7 kB/s eta 0:00:01
           -------------------------------- ----- 122.9/145.4 kB 1.4 MB/s eta 0:00:01
           -------------------------------------- 145.4/145.4 kB 1.4 MB/s eta 0:00:00
        Installing build dependencies: started
        Installing build dependencies: finished with status 'done'
        Getting requirements to build wheel: started
        Getting requirements to build wheel: finished with status 'done'
        Preparing metadata (pyproject.toml): started
        Preparing metadata (pyproject.toml): finished with status 'error'
        error: subprocess-exited-with-error
      
        Ã— Preparing metadata (pyproject.toml) did not run successfully.
        â”

In [4]:
import pandas as pd
import pymongo
from pathlib import Path

# establishing data

In [22]:
sources = {'mat':'student-mat.csv', 'por': 'student-por.csv'}

grades_df = pd.DataFrame()

for k, v in sources.items():
    df = pd.read_csv(f'data/{v}', delimiter=';')
    df['subj'] = k
    new_cols = []
    for col in df.columns:
        if col == col.lower():
            new_cols.append(col)
        else:
            if col[0] == col[0].upper():
                new_cols.append(col[0].lower() + "_" + col[1:])
    df.columns = new_cols
    grades_df = pd.concat([grades_df, df], ignore_index=True)


In [23]:
grades_df.reindex()

,school,sex,age,address,famsize,p_status,m_edu,f_edu,m_job,f_job,...,freetime,goout,d_alc,w_alc,health,absences,g_1,g_2,g_3,subj
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,3,4,1,1,3,6,5,6,6,mat
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,3,3,1,1,3,4,5,5,6,mat
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,3,2,2,3,3,10,7,8,10,mat
3,GP,F,15,U,GT3,T,4,2,health,services,...,2,2,1,1,5,2,15,14,15,mat
4,GP,F,16,U,GT3,T,3,3,other,other,...,3,2,1,2,5,4,6,10,10,mat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1039,MS,F,19,R,GT3,T,2,3,services,other,...,4,2,1,2,5,4,10,11,10,por
1040,MS,F,18,U,LE3,T,3,1,teacher,services,...,3,4,1,1,1,4,15,15,16,por
1041,MS,F,18,U,GT3,T,1,1,other,other,...,1,1,1,1,5,6,11,12,9,por
1042,MS,M,17,U,LE3,T,3,1,services,services,...,4,5,3,4,2,6,10,10,10,por


In [30]:
grades_df[].head()

,school,sex,age,address,famsize,p_status,m_edu,f_edu,m_job,f_job,...,freetime,goout,d_alc,w_alc,health,absences,g_1,g_2,g_3,subj
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,3,4,1,1,3,6,5,6,6,mat
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,3,3,1,1,3,4,5,5,6,mat
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,3,2,2,3,3,10,7,8,10,mat
3,GP,F,15,U,GT3,T,4,2,health,services,...,2,2,1,1,5,2,15,14,15,mat
4,GP,F,16,U,GT3,T,3,3,other,other,...,3,2,1,2,5,4,6,10,10,mat


In [33]:
sample = grades_df.loc[0]
bool_cols = []

for i in range(len(sample)):
    try:
        if sample.iloc[i].lower() == 'yes' or sample.iloc[i].lower() == 'no':
            bool_cols.append(grades_df.columns[i])
    except:
        pass


<class 'pandas.core.series.Series'>


['schoolsup',
 'famsup',
 'paid',
 'activities',
 'nursery',
 'higher',
 'internet',
 'romantic']

In [34]:
for col in bool_cols:
    grades_df[col] = grades_df[col].str.replace('yes','True')
    grades_df[col] = grades_df[col].str.replace('no', 'False')
    grades_df[col] = grades_df[col].astype('bool')

In [43]:
print(grades_df.memory_usage().sum())

grades_df.info()

225636
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1044 entries, 0 to 1043
Data columns (total 34 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   school      1044 non-null   object
 1   sex         1044 non-null   object
 2   age         1044 non-null   int64 
 3   address     1044 non-null   object
 4   famsize     1044 non-null   object
 5   p_status    1044 non-null   object
 6   m_edu       1044 non-null   int64 
 7   f_edu       1044 non-null   int64 
 8   m_job       1044 non-null   object
 9   f_job       1044 non-null   object
 10  reason      1044 non-null   object
 11  guardian    1044 non-null   object
 12  traveltime  1044 non-null   int64 
 13  studytime   1044 non-null   int64 
 14  failures    1044 non-null   int64 
 15  schoolsup   1044 non-null   bool  
 16  famsup      1044 non-null   bool  
 17  paid        1044 non-null   bool  
 18  activities  1044 non-null   bool  
 19  nursery     1044 non-null   bool  
 20  h

In [45]:
grades_df.to_dict(orient='records')

[{'school': 'GP',
  'sex': 'F',
  'age': 18,
  'address': 'U',
  'famsize': 'GT3',
  'p_status': 'A',
  'm_edu': 4,
  'f_edu': 4,
  'm_job': 'at_home',
  'f_job': 'teacher',
  'reason': 'course',
  'guardian': 'mother',
  'traveltime': 2,
  'studytime': 2,
  'failures': 0,
  'schoolsup': True,
  'famsup': True,
  'paid': True,
  'activities': True,
  'nursery': True,
  'higher': True,
  'internet': True,
  'romantic': True,
  'famrel': 4,
  'freetime': 3,
  'goout': 4,
  'd_alc': 1,
  'w_alc': 1,
  'health': 3,
  'absences': 6,
  'g_1': 5,
  'g_2': 6,
  'g_3': 6,
  'subj': 'mat'},
 {'school': 'GP',
  'sex': 'F',
  'age': 17,
  'address': 'U',
  'famsize': 'GT3',
  'p_status': 'T',
  'm_edu': 1,
  'f_edu': 1,
  'm_job': 'at_home',
  'f_job': 'other',
  'reason': 'course',
  'guardian': 'father',
  'traveltime': 1,
  'studytime': 2,
  'failures': 0,
  'schoolsup': True,
  'famsup': True,
  'paid': True,
  'activities': True,
  'nursery': True,
  'higher': True,
  'internet': True,
  'rom